<a href="https://colab.research.google.com/github/heyrobin/Python/blob/main/Bitcoin_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#importing libraries
import tweepy
from textblob import TextBlob
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [23]:
#importing keys to authenticate the access twitter API
#connect as OATH handler
consumer_key = "8AO6OU5ubyi4XO47b1C7Sjdlz"
consumer_sec = "FS1usPrfPolvjLXbwGka5N8TWkOZhUsdxGmmTwuO016koesUSt"

# from proxy server we need to connect
access_token = "1151573806680592384-OUFeUtpsRFZM6jQxl1AG99NEjlY0Kt"
access_token_sec = "KKHmkHkDGVaDof8XK4fKKI52DmNl4vZlaXnx85WRfd4Lr"


In [24]:
#explore tweepy
dir(tweepy)

['API',
 'AppAuthHandler',
 'Cache',
 'Cursor',
 'DirectMessage',
 'FileCache',
 'Friendship',
 'MemoryCache',
 'ModelFactory',
 'OAuthHandler',
 'RateLimitError',
 'SavedSearch',
 'SearchResults',
 'Status',
 'Stream',
 'StreamListener',
 'TweepError',
 'User',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__license__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'api',
 'auth',
 'binder',
 'cache',
 'cursor',
 'debug',
 'error',
 'models',
 'parsers',
 'streaming',
 'utils']

In [ ]:
#creating authentication object
authenticate = tweepy.OAuthHandler(consumer_key,consumer_sec)
#set the access token and the access token secret
authenticate.set_access_token(access_token,access_token_sec)
#create the API object
api = tweepy.API(authenticate)

In [ ]:
#Gather 1000 tweets about cardano and filter out any retweets
search_term = '#cardano -filter:retweets'
#Create a cursor object
tweets = tweepy.Cursor(api.search, q=search_term, lang='en', since='2021-01-01',tweet_mode= 'extended').items(2000)
#store the tweets in variable and get the full text
all_tweets = [tweet.full_text for tweet in tweets]

In [ ]:
#create a dataframe to store the tweets with a column called 'Tweets'
df = pd.DataFrame(all_tweets, columns=['Tweets'])
df.head(10)

,Tweets
0,Cardano (ADA) Price Analysis: Trading Near Cru...
1,I just sold all of my #Cardano #ADA for the be...
2,We are working with the Ethiopian government t...
3,Is it your time dogecoin omg its going up \n#d...
4,Cardano ADA Current Price:\n$1.1887\n1 Hour: -...
5,Its going up dogecoin go look \n#dogecoin #BAT...
6,Elon musk its your time to move on dogecoin lo...
7,"PMLX was there with ♥️, proudly under @IOHK_Ch..."
8,Did you buy dogecoin go go go go now!!!!!!\n#d...
9,@WiseCapital3 @Immortalxplorer Who is spreadin...


In [ ]:
#Create a function to clean the tweets
def cleanTwt(twt):
  twt = re.sub('#cardano', 'cardano', twt)
  twt = re.sub('#Cardano', 'Cardano', twt)
  twt = re.sub('#[A-Za-z0-9]+', '', twt)
  twt = re.sub('\\n', '', twt)
  twt = re.sub('https?:\/\/\S+','', twt) #removes any hyperlinks
  return twt


In [ ]:
#Clean the tweets
df['Cleaned_Tweets'] = df['Tweets'].apply(cleanTwt)
#result
df.head(10)

,Tweets,Cleaned_Tweets
0,Cardano (ADA) Price Analysis: Trading Near Cru...,Cardano (ADA) Price Analysis: Trading Near Cru...
1,I just sold all of my #Cardano #ADA for the be...,"I just sold all of my Cardano for the better,..."
2,We are working with the Ethiopian government t...,We are working with the Ethiopian government t...
3,Is it your time dogecoin omg its going up \n#d...,Is it your time dogecoin omg its going up C...
4,Cardano ADA Current Price:\n$1.1887\n1 Hour: -...,Cardano ADA Current Price:$1.18871 Hour: -0.81...
5,Its going up dogecoin go look \n#dogecoin #BAT...,Its going up dogecoin go look Cardano ...
6,Elon musk its your time to move on dogecoin lo...,Elon musk its your time to move on dogecoin lo...
7,"PMLX was there with ♥️, proudly under @IOHK_Ch...","PMLX was there with ♥️, proudly under @IOHK_Ch..."
8,Did you buy dogecoin go go go go now!!!!!!\n#d...,Did you buy dogecoin go go go go now!!!!!! C...
9,@WiseCapital3 @Immortalxplorer Who is spreadin...,@WiseCapital3 @Immortalxplorer Who is spreadin...


In [ ]:
#Create a function to get the subjectivity
def getSubjectivity(twt):